# Sales Analysis

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Number of rows
n_orders = 10000

# Generate data
data = {
    "OrderID": [f"ORD-{i}" for i in range(1, n_orders + 1)],
    "CustomerID": [f"CUST-{np.random.randint(1, 2001)}" for _ in range(n_orders)],
    "ProductID": [f"PROD-{np.random.randint(1, 501)}" for _ in range(n_orders)],
    "OrderDate": [datetime(2022, 1, 1) + timedelta(days=np.random.randint(0, 365)) for _ in range(n_orders)],
    "ShipDate": [datetime(2022, 1, 1) + timedelta(days=np.random.randint(1, 10)) for _ in range(n_orders)],
    "Category": np.random.choice(["Electronics", "Clothing", "Home Appliances", "Books"], n_orders),
    "SubCategory": np.random.choice(["Smartphones", "Laptops", "Shirts", "Kitchenware", "Novels"], n_orders),
    "Region": np.random.choice(["North America", "Europe", "Asia", "South America"], n_orders),
    "Quantity": np.random.randint(1, 20, n_orders),
    "UnitPrice": np.round(np.random.uniform(10, 500, n_orders), 2),
    "Discount": np.round(np.random.uniform(0, 30, n_orders), 2),
    "ShippingCost": np.round(np.random.uniform(5, 50, n_orders), 2),
}

# Calculate TotalSales and Profit
df = pd.DataFrame(data)
df["TotalSales"] = np.round(df["Quantity"] * df["UnitPrice"] * (1 - df["Discount"] / 100), 2)
df["Profit"] = np.round(df["TotalSales"] - (df["Quantity"] * df["UnitPrice"] * 0.6 + df["ShippingCost"]), 2)

# Display first few rows
print(df.head())

# Save to CSV
output_file = "sales_data.csv"
df.to_csv(output_file, index=False)
print(f"Dataset saved to {output_file}")

  OrderID CustomerID ProductID  OrderDate   ShipDate         Category  \
0   ORD-1  CUST-1127  PROD-229 2022-11-17 2022-01-06            Books   
1   ORD-2  CUST-1460  PROD-350 2022-04-18 2022-01-06  Home Appliances   
2   ORD-3   CUST-861  PROD-275 2022-10-03 2022-01-02  Home Appliances   
3   ORD-4  CUST-1295  PROD-442 2022-06-29 2022-01-05  Home Appliances   
4   ORD-5  CUST-1131  PROD-279 2022-08-01 2022-01-02         Clothing   

   SubCategory         Region  Quantity  UnitPrice  Discount  ShippingCost  \
0       Novels         Europe        19      64.14     27.77         34.51   
1  Kitchenware  North America        16     428.01     21.73         47.46   
2       Novels  South America         2      94.62     26.03         17.54   
3       Shirts           Asia        15     363.26     19.12         26.23   
4  Smartphones  South America         9      30.87     18.77         38.45   

   TotalSales   Profit  
0      880.24   114.53  
1     5360.05  1203.69  
2      139.98    

In [2]:
# Create a dictionary to store all KPI results
kpi_results = {}

In [3]:
# 1. Total Revenue
kpi_results["Total Revenue"] = f"${df['TotalSales'].sum():,.2f}"
print(f"Total Revenue: {kpi_results['Total Revenue']}")

Total Revenue: $21,860,909.95


In [4]:
# 2. Average Order Value (AOV)
kpi_results["Average Order Value (AOV)"] = f"${df['TotalSales'].mean():,.2f}"
print(f"Average Order Value (AOV): {kpi_results['Average Order Value (AOV)']}")

Average Order Value (AOV): $2,186.09


In [5]:
# 3. Profit Margin
profit_margin = (df["Profit"].sum() / df["TotalSales"].sum()) * 100
kpi_results["Profit Margin"] = f"{profit_margin:.2f}%"
print(f"Profit Margin: {kpi_results['Profit Margin']}")

Profit Margin: 28.22%


In [6]:
# 4. Top-Performing Product
top_product = df.groupby("ProductID")["TotalSales"].sum().idxmax()
top_product_sales = df.groupby("ProductID")["TotalSales"].sum().max()
kpi_results["Top-Performing Product"] = f"{top_product} with Sales ${top_product_sales:,.2f}"
print(f"Top-Performing Product: {kpi_results['Top-Performing Product']}")


Top-Performing Product: PROD-77 with Sales $83,802.81


In [7]:
# 5. Customer Lifetime Value (CLV)
clv = df.groupby("CustomerID")["TotalSales"].sum().sort_values(ascending=False).head()
clv_csv = clv.reset_index()
clv_csv.columns = ["CustomerID", "TotalSales"]
print(clv)


CustomerID
CUST-1800    39104.84
CUST-685     34176.90
CUST-1257    33221.90
CUST-1261    32742.02
CUST-1953    32361.54
Name: TotalSales, dtype: float64


In [8]:
# 6. Category Revenue Distribution
category_revenue = df.groupby("Category")["TotalSales"].sum()
category_revenue_percentage = (category_revenue / df["TotalSales"].sum()) * 100
kpi_results["Category Revenue Distribution"] = category_revenue_percentage.to_dict()
print(category_revenue_percentage)


Category
Books              25.215970
Clothing           24.868947
Electronics        24.476043
Home Appliances    25.439040
Name: TotalSales, dtype: float64


In [9]:
# 7. Region Performance
region_performance = df.groupby("Region")[["TotalSales", "Profit"]].sum()
kpi_results["Region Performance"] = region_performance.to_dict()
print(region_performance)


               TotalSales      Profit
Region                               
Asia           5483678.31  1537789.71
Europe         5514968.17  1556859.46
North America  5483025.74  1553997.54
South America  5379237.73  1520630.58


In [10]:
# 8. Discount Effectiveness
discount_correlation = df["Discount"].corr(df["Quantity"])
kpi_results["Discount Effectiveness"] = f"{discount_correlation:.2f}"
print(f"Discount Effectiveness (Correlation between Discount and Quantity): {discount_correlation:.2f}")


Discount Effectiveness (Correlation between Discount and Quantity): 0.00


In [11]:
# 9. Shipping Cost Impact
shipping_cost_impact = (df["ShippingCost"].sum() / df["TotalSales"].sum()) * 100
kpi_results["Shipping Cost Impact"] = f"{shipping_cost_impact:.2f}%"
print(f"Shipping Cost Impact: {shipping_cost_impact:.2f}%")


Shipping Cost Impact: 1.26%


In [12]:
# 10. Average Fulfillment Time
df["FulfillmentTime"] = (df["ShipDate"] - df["OrderDate"]).dt.days
avg_fulfillment_time = df["FulfillmentTime"].mean()
kpi_results["Average Fulfillment Time"] = f"{avg_fulfillment_time:.2f} days"
print(f"Average Fulfillment Time: {avg_fulfillment_time:.2f} days")

Average Fulfillment Time: -176.74 days


In [13]:
# 11. Revenue-Weighted Profitability by Customer Segment
segment_performance = df.groupby("Region").agg({
    "TotalSales": "sum",
    "Profit": "sum"
})
segment_performance["Profitability"] = segment_performance["Profit"] / segment_performance["TotalSales"] * 100
segment_performance["RevenueWeight"] = segment_performance["TotalSales"] / df["TotalSales"].sum() * 100
segment_performance["WeightedProfitability"] = segment_performance["Profitability"] * segment_performance["RevenueWeight"] / 100
kpi_results["Revenue-Weighted Profitability"] = segment_performance[["Profitability", "RevenueWeight", "WeightedProfitability"]].to_dict()
print(f"Revenue-Weighted Profitability: {kpi_results['Revenue-Weighted Profitability']}")

Revenue-Weighted Profitability: {'Profitability': {'Asia': 28.043032852523403, 'Europe': 28.22970889422196, 'North America': 28.34197054125082, 'South America': 28.26851417105895}, 'RevenueWeight': {'Asia': 25.084400981213495, 'Europe': 25.22753253461894, 'North America': 25.081415881318332, 'South America': 24.606650602849218}, 'WeightedProfitability': {'Asia': 7.034426808020402, 'Europe': 7.121658995718062, 'North America': 7.108567500411847, 'South America': 6.955934512689393}}


In [14]:
# 12. Repeat Purchase Rate
repeat_customers = df["CustomerID"].value_counts()
repeat_purchase_rate = (repeat_customers[repeat_customers > 1].count() / repeat_customers.count()) * 100
kpi_results["Repeat Purchase Rate"] = f"{repeat_purchase_rate:.2f}%"
print(f"Repeat Purchase Rate: {kpi_results['Repeat Purchase Rate']}")

Repeat Purchase Rate: 96.28%


In [15]:
# 13. Customer Churn Rate
customer_orders = df.groupby("CustomerID").agg({
    "OrderDate": "count",
    "TotalSales": "sum"
}).rename(columns={"OrderDate": "OrderCount", "TotalSales": "TotalRevenue"})
customer_orders["RevenuePerOrder"] = customer_orders["TotalRevenue"] / customer_orders["OrderCount"]
churn_customers = customer_orders[customer_orders["OrderCount"] < 2]
churn_rate = (churn_customers.shape[0] / customer_orders.shape[0]) * 100
kpi_results["Customer Churn Rate"] = f"{churn_rate:.2f}%"
print(f"Customer Churn Rate: {kpi_results['Customer Churn Rate']}")

Customer Churn Rate: 3.72%


In [16]:
# 14. Time Decay Revenue Trends
def time_decay_revenue(order_date, sales, decay_factor=0.9):
    days_from_last_order = (df["OrderDate"].max() - order_date).dt.days
    return sales * (decay_factor ** days_from_last_order)

df["DecayWeightedSales"] = time_decay_revenue(df["OrderDate"], df["TotalSales"])
time_decay_revenue_trend = df.groupby(df["OrderDate"].dt.month)["DecayWeightedSales"].sum()
kpi_results["Time Decay Revenue Trend"] = time_decay_revenue_trend.to_dict()
print(f"Time Decay Revenue Trend: {kpi_results['Time Decay Revenue Trend']}")

Time Decay Revenue Trend: {1: 3.0993314856672105e-10, 2: 5.7316512157578305e-09, 3: 1.5826329520482461e-07, 4: 3.418319612201721e-06, 5: 9.812627571762682e-05, 6: 0.0020611154132706335, 7: 0.05784092322017145, 8: 1.509281981891807, 9: 32.90379386553637, 10: 883.5733784940035, 11: 23147.683147051368, 12: 535946.6576839592}


In [17]:
# 15. Inventory Risk by Subcategory
stock_data = df.groupby("SubCategory")["Quantity"].sum()
initial_stock = 100 * df["SubCategory"].nunique()
inventory_risk = initial_stock - stock_data
kpi_results["Inventory Risk by Subcategory"] = inventory_risk.to_dict()
print(f"Inventory Risk by Subcategory: {kpi_results['Inventory Risk by Subcategory']}")

Inventory Risk by Subcategory: {'Kitchenware': -21056, 'Laptops': -20222, 'Novels': -19082, 'Shirts': -18600, 'Smartphones': -18879}


In [18]:
# 16. Profit Leakage
df["DiscountImpact"] = df["Quantity"] * df["UnitPrice"] * (df["Discount"] / 100)
df["ProfitLeakage"] = df["DiscountImpact"] + df["ShippingCost"]
total_leakage = df["ProfitLeakage"].sum()
kpi_results["Total Profit Leakage"] = f"${total_leakage:,.2f}"
print(f"Total Profit Leakage: {kpi_results['Total Profit Leakage']}")

Total Profit Leakage: $4,107,987.53


In [19]:
# 17. Customer Dependency Risk
top_5_percent_customers = int(0.05 * df["CustomerID"].nunique())
top_customers_revenue = df.groupby("CustomerID")["TotalSales"].sum().nlargest(top_5_percent_customers).sum()
dependency_risk = (top_customers_revenue / df["TotalSales"].sum()) * 100
kpi_results["Customer Dependency Risk"] = f"{dependency_risk:.2f}%"
print(f"Customer Dependency Risk: {kpi_results['Customer Dependency Risk']}")

Customer Dependency Risk: 11.93%


In [20]:
# 18. Cost-to-Serve Index
region_cost_to_serve = df.groupby("Region").agg({
    "ShippingCost": "sum",
    "DiscountImpact": "sum",
    "TotalSales": "sum"
})
region_cost_to_serve["CostToServeIndex"] = (region_cost_to_serve["ShippingCost"] + region_cost_to_serve["DiscountImpact"]) / region_cost_to_serve["TotalSales"] * 100
kpi_results["Cost-to-Serve Index"] = region_cost_to_serve["CostToServeIndex"].to_dict()
print(f"Cost-to-Serve Index: {kpi_results['Cost-to-Serve Index']}")

Cost-to-Serve Index: {'Asia': 19.08804349661058, 'Europe': 18.774540578735564, 'North America': 18.58224668224884, 'South America': 18.719777322055627}


In [21]:
# 19. Cross-Sell Opportunity Score
cross_sell_score = df.groupby("CustomerID")["Category"].nunique().mean()
kpi_results["Cross-Sell Opportunity Score"] = f"{cross_sell_score:.2f}"
print(f"Cross-Sell Opportunity Score: {kpi_results['Cross-Sell Opportunity Score']}")

Cross-Sell Opportunity Score: 2.86


In [22]:
# 20. Seasonal Sales Sensitivity
df["Month"] = df["OrderDate"].dt.month
seasonal_sales = df.groupby(["Month", "Category"])["TotalSales"].sum().unstack()
kpi_results["Seasonal Sales Sensitivity"] = seasonal_sales.to_dict()
print(f"Seasonal Sales Sensitivity: {kpi_results['Seasonal Sales Sensitivity']}")

Seasonal Sales Sensitivity: {'Books': {1: 517738.54, 2: 443941.7, 3: 441536.17, 4: 424375.6, 5: 519235.58, 6: 449500.99, 7: 446162.25, 8: 443458.06, 9: 437129.94, 10: 462703.87, 11: 497590.18, 12: 429067.69}, 'Clothing': {1: 406832.51, 2: 411949.71, 3: 486005.11, 4: 387701.69, 5: 440744.62, 6: 480010.47, 7: 561190.35, 8: 500007.61, 9: 431425.59, 10: 393290.07, 11: 449365.14, 12: 488055.14}, 'Electronics': {1: 470013.21, 2: 435174.88, 3: 493101.77, 4: 445353.59, 5: 467604.87, 6: 381611.32, 7: 486347.05, 8: 492502.46, 9: 434195.6, 10: 428343.61, 11: 421287.55, 12: 395149.76}, 'Home Appliances': {1: 481606.59, 2: 417128.15, 3: 481256.84, 4: 442714.61, 5: 489573.87, 6: 433401.99, 7: 439591.89, 8: 499756.04, 9: 439056.64, 10: 495307.73, 11: 439196.65, 12: 502614.7}}


In [23]:
# Save all outputs to CSV
kpi_output = pd.DataFrame(list(kpi_results.items()), columns=["KPI", "Value"])
kpi_output.to_csv("all_kpi_results.csv", index=False)

print("All KPI results saved to 'all_kpi_results.csv'")

All KPI results saved to 'all_kpi_results.csv'
